In [1]:
import numpy as np
import torch
from deepymod_torch.library_function import library_new, Library
from deepymod_torch.DeepMod import DeepMod, deepmod_type
from deepymod_torch.sparsity import scaling, threshold
from deepymod_torch.nn import create_deriv_data

import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')

np.random.seed(42)

%load_ext autoreload
%autoreload 2

#torch.set_default_tensor_type('torch.cuda.FloatTensor')  # enable for gpu.

In [2]:
data = np.load('../data/processed/burgers.npy', allow_pickle=True).item()

In [3]:
X = np.transpose((data['t'].flatten(), data['x'].flatten()))
y = np.real(data['u']).reshape((data['u'].size, 1))

In [6]:
number_of_samples = 500

idx = np.random.permutation(y.size)
X_train = create_deriv_data(torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32, requires_grad=True), 2)
y_train = torch.tensor(y[idx, :][:number_of_samples], dtype=torch.float32)

In [7]:
config = {'input_dim': 2, 'hidden_dim': 20, 'layers': 5, 'output_dim': 1, 'library_function': Library, 'library_args':{'poly_order': 2, 'diff_order': 2}}
model = DeepMod(config)

In [8]:
len(model(X_train))

3

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)  

In [ ]:
%%time
model.train(X_train, y_train, optimizer, 5000)

| Iteration | Progress | Time remaining |     Cost |      MSE |      Reg |       L1 |
       4800     96.00%               0s   1.32e-04   0.00e+00   0.00e+00   0.00e+00 

In [10]:
model.coeff_vector_list[0]

Parameter containing:
tensor([[-8.2583e-05],
        [-3.3998e-02],
        [ 8.3280e-02],
        [-4.0696e-04],
        [-8.4463e-01],
        [ 4.8189e-04],
        [-7.2691e-03],
        [-8.7235e-02],
        [ 1.1120e-05]], requires_grad=True)

In [28]:
torch.get_num_threads()

7